In [ ]:
import requests
import json
from google_trans_new import google_translator
translator = google_translator()
# 請將 YOUR_API_KEY 替換為你申請的 API 金鑰
API_KEY = 'UHIEW0wdDSTLBdAGzU5w9bssk8eGgK40FODJIsSY'
BASE_URL = 'https://api.nal.usda.gov/fdc/v1/'


fdc_id = 2727568  # Apple, raw 的 FDC ID

def get_food_details(fdc_id):
    url = f"{BASE_URL}food/{fdc_id}?api_key={API_KEY}"
    response = requests.get(url)
    return response.json()

food_json = get_food_details(fdc_id)
; print(food_json)
; print("食物名稱:", food_json.get('servingSize'))

if food_json.get('servingSize') is None:
    # 代表這筆資料就是 100g 基準
    factor = 1
else:
    # 如果有份量，需要換算到 100g
    factor = 100 / food_json['servingSize']

{'fdcId': 2727568, 'description': 'Chicken, wing, meat and skin, raw', 'publicationDate': '4/24/2025', 'foodNutrients': [{'nutrient': {'id': 2045, 'number': '951', 'name': 'Proximates', 'rank': 50, 'unitName': 'g'}, 'type': 'FoodNutrient'}, {'type': 'FoodNutrient', 'nutrient': {'id': 1051, 'number': '255', 'name': 'Water', 'rank': 100, 'unitName': 'g'}, 'foodNutrientDerivation': {'id': 1, 'code': 'A', 'description': 'Analytical', 'foodNutrientSource': {'id': 1, 'code': '1', 'description': 'Analytical or derived from analytical'}}, 'id': 34678478, 'amount': 70.66, 'dataPoints': 8, 'max': 71.51, 'min': 69.38, 'median': 70.91, 'minYearAcquired': 2024, 'nutrientAnalysisDetails': [{'subSampleId': 2728823, 'nutrientId': 1051, 'nutrientAcquisitionDetails': [{'sampleUnitId': 2727622, 'purchaseDate': '9/22/2024', 'storeCity': 'Hunt Valley', 'storeState': 'MD', 'packerCity': 'Rochester', 'packerState': 'NY'}], 'amount': 70.7, 'labMethodTechnique': 'Vacuum oven', 'labMethodDescription': 'AOAC 934

In [8]:
#修改範例程式
def parse_nutrients(food_json):
    nutrient_map = {
        "Energy": "energy_kcal",
        "Protein": "protein",
        "Total lipid (fat)": "fat",
        "Carbohydrate, by difference": "carbs",
        "Sugars, total including NLEA": "sugar",
        "Sodium, Na": "sodium",
        "Water": "water"
    }

    data = {}

    for nutrient in food_json.get('foodNutrients', []):
        # 適配新版 API
        if 'nutrient' in nutrient:
            name = nutrient['nutrient'].get('name')
            value = nutrient.get('amount')
        else:
            name = nutrient.get('nutrientName')
            value = nutrient.get('value')

        if name in nutrient_map:
            if name == "Sodium, Na" and value is not None:
                value = value / 1000  # mg -> g
            data[nutrient_map[name]] = value

    return data

# 範例抓取一個食物
fdc_id = 1102645  # Apple, raw 的 FDC ID

nutrients = parse_nutrients(food_json)
ingredient_name = food_json['description']
c_name =ingredient_name
# 生成 MySQL INSERT 語句
insert_sql = f"""
INSERT INTO ingredient (ingredient_name, c_name, energy_kcal, protein, fat, carbs, sugar, sodium, water)
VALUES ('{ingredient_name}', '{c_name}', {nutrients.get('energy_kcal', 'NULL')}, {nutrients.get('protein', 'NULL')}, 
        {nutrients.get('fat', 'NULL')}, {nutrients.get('carbs', 'NULL')}, {nutrients.get('sugar', 'NULL')}, 
        {nutrients.get('sodium', 'NULL')}, {nutrients.get('water', 'NULL')});
"""
print(insert_sql)



INSERT INTO ingredient (ingredient_name, c_name, energy_kcal, protein, fat, carbs, sugar, sodium, water)
VALUES ('Applesauce, regular', 'Applesauce, regular', 68, 0.16, 
        0.17, 17.49, NULL, 
        0.002, 81.98);



In [ ]:
{'wweiaFoodCategory': {'wweiaFoodCategoryCode': 6002, 'wweiaFoodCategoryDescription': 'Apples'}, 'description': 'Applesauce, regular', 'foodAttributes': [{'id': 998364, 'value': 'Apples', 'name': 'WWEIA Category description', 'foodAttributeType': {'id': 999, 'name': 'Attribute', 'description': 'Generic attributes'}}, {'id': 990173, 'value': '6002', 'name': 'WWEIA Category number', 'foodAttributeType': {'id': 999, 'name': 'Attribute', 'description': 'Generic attributes'}}, {'id': 976016, 'value': 'applesauce, NFS', 'sequenceNumber': 2, 'foodAttributeType': {'id': 1001, 'name': 'Additional Description', 'description': 'Additional descriptions for the food.'}}], 'foodCode': '63101110', 'inputFoods': [{'id': 98321, 'foodDescription': 'Applesauce, canned, sweetened, without salt', 'ingredientDescription': 'Applesauce, canned, sweetened, without salt', 'ingredientWeight': 100.0, 'portionCode': '0', 'portionDescription': 'NONE', 'sequenceNumber': 1, 'ingredientCode': 9020, 'retentionCode': 0, 'unit': 'GM', 'amount': 100.0}], 'startDate': '1/1/2017', 'endDate': '12/31/2018', 'foodComponents': [], 'foodClass': 'Survey', 'fdcId': 1102645, 'publicationDate': '10/30/2020', 'foodNutrients': [{'nutrient': {'id': 2045, 'number': '951', 'name': 'Proximates', 'rank': 50, 'unitName': 'g'}, 'type': 'FoodNutrient'}, {'type': 'FoodNutrient', 'nutrient': {'id': 1051, 'number': '255', 'name': 'Water', 'rank': 100, 'unitName': 'g'}, 'id': 13208998, 'amount': 81.98}, {'type': 'FoodNutrient', 'nutrient': {'id': 1008, 'number': '208', 'name': 'Energy', 'rank': 300, 'unitName': 'kcal'}, 'id': 13208996, 'amount': 68}, {'type': 'FoodNutrient', 'nutrient': {'id': 1003, 'number': '203', 'name': 'Protein', 'rank': 600, 'unitName': 'g'}, 'id': 13208993, 'amount': 0.16}, {'type': 'FoodNutrient', 'nutrient': {'id': 1004, 'number': '204', 'name': 'Total lipid (fat)', 'rank': 800, 'unitName': 'g'}, 'id': 13208994, 'amount': 0.17}, {'nutrient': {'id': 2039, 'number': '956', 'name': 'Carbohydrates', 'rank': 1100, 'unitName': 'g'}, 'type': 'FoodNutrient'}, {'type': 'FoodNutrient', 'nutrient': {'id': 1005, 'number': '205', 'name': 'Carbohydrate, by difference', 'rank': 1110, 'unitName': 'g'}, 'id': 13208995, 'amount': 17.49}, {'type': 'FoodNutrient', 'nutrient': {'id': 1079, 'number': '291', 'name': 'Fiber, total dietary', 'rank': 1200, 'unitName': 'g'}, 'id': 13209002, 'amount': 1.2}, {'type': 'FoodNutrient', 'nutrient': {'id': 2000, 'number': '269', 'name': 'Total Sugars', 'rank': 1510, 'unitName': 'g'}, 'id': 13209001, 'amount': 14.67}, {'nutrient': {'id': 2043, 'number': '300', 'name': 'Minerals', 'rank': 5200, 'unitName': 'mg'}, 'type': 'FoodNutrient'}, {'type': 'FoodNutrient', 'nutrient': {'id': 1087, 'number': '301', 'name': 'Calcium, Ca', 'rank': 5300, 'unitName': 'mg'}, 'id': 13209003, 'amount': 3}, {'type': 'FoodNutrient', 'nutrient': {'id': 1089, 'number': '303', 'name': 'Iron, Fe', 'rank': 5400, 'unitName': 'mg'}, 'id': 13209004, 'amount': 0.12}, {'type': 'FoodNutrient', 'nutrient': {'id': 1090, 'number': '304', 'name': 'Magnesium, Mg', 'rank': 5500, 'unitName': 'mg'}, 'id': 13209005, 'amount': 3}, {'type': 'FoodNutrient', 'nutrient': {'id': 1091, 'number': '305', 'name': 'Phosphorus, P', 'rank': 5600, 'unitName': 'mg'}, 'id': 13209006, 'amount': 6}, {'type': 'FoodNutrient', 'nutrient': {'id': 1092, 'number': '306', 'name': 'Potassium, K', 'rank': 5700, 'unitName': 'mg'}, 'id': 13209007, 'amount': 75}, {'type': 'FoodNutrient', 'nutrient': {'id': 1093, 'number': '307', 'name': 'Sodium, Na', 'rank': 5800, 'unitName': 'mg'}, 'id': 13209008, 'amount': 2}, {'type': 'FoodNutrient', 'nutrient': {'id': 1095, 'number': '309', 'name': 'Zinc, Zn', 'rank': 5900, 'unitName': 'mg'}, 'id': 13209009, 'amount': 0.03}, {'type': 'FoodNutrient', 'nutrient': {'id': 1098, 'number': '312', 'name': 'Copper, Cu', 'rank': 6000, 'unitName': 'mg'}, 'id': 13209010, 'amount': 0.032}, {'type': 'FoodNutrient', 'nutrient': {'id': 1103, 'number': '317', 'name': 'Selenium, Se', 'rank': 6200, 'unitName': 'µg'}, 'id': 13209011, 'amount': 0.3}, {'nutrient': {'id': 2046, 'number': '952', 'name': 'Vitamins and Other Components', 'rank': 6250, 'unitName': 'g'}, 'type': 'FoodNutrient'}, {'type': 'FoodNutrient', 'nutrient': {'id': 1162, 'number': '401', 'name': 'Vitamin C, total ascorbic acid', 'rank': 6300, 'unitName': 'mg'}, 'id': 13209021, 'amount': 1.7}, {'type': 'FoodNutrient', 'nutrient': {'id': 1165, 'number': '404', 'name': 'Thiamin', 'rank': 6400, 'unitName': 'mg'}, 'id': 13209022, 'amount': 0.017}, {'type': 'FoodNutrient', 'nutrient': {'id': 1166, 'number': '405', 'name': 'Riboflavin', 'rank': 6500, 'unitName': 'mg'}, 'id': 13209023, 'amount': 0.022}, {'type': 'FoodNutrient', 'nutrient': {'id': 1167, 'number': '406', 'name': 'Niacin', 'rank': 6600, 'unitName': 'mg'}, 'id': 13209024, 'amount': 0.072}, {'type': 'FoodNutrient', 'nutrient': {'id': 1175, 'number': '415', 'name': 'Vitamin B-6', 'rank': 6800, 'unitName': 'mg'}, 'id': 13209025, 'amount': 0.027}, {'type': 'FoodNutrient', 'nutrient': {'id': 1177, 'number': '417', 'name': 'Folate, total', 'rank': 6900, 'unitName': 'µg'}, 'id': 13209026, 'amount': 1}, {'type': 'FoodNutrient', 'nutrient': {'id': 1186, 'number': '431', 'name': 'Folic acid', 'rank': 7000, 'unitName': 'µg'}, 'id': 13209030, 'amount': 0}, {'type': 'FoodNutrient', 'nutrient': {'id': 1187, 'number': '432', 'name': 'Folate, food', 'rank': 7100, 'unitName': 'µg'}, 'id': 13209031, 'amount': 1}, {'type': 'FoodNutrient', 'nutrient': {'id': 1190, 'number': '435', 'name': 'Folate, DFE', 'rank': 7200, 'unitName': 'µg'}, 'id': 13209032, 'amount': 1}, {'type': 'FoodNutrient', 'nutrient': {'id': 1180, 'number': '421', 'name': 'Choline, total', 'rank': 7220, 'unitName': 'mg'}, 'id': 13209028, 'amount': 3.3}, {'type': 'FoodNutrient', 'nutrient': {'id': 1178, 'number': '418', 'name': 'Vitamin B-12', 'rank': 7300, 'unitName': 'µg'}, 'id': 13209027, 'amount': 0}, {'type': 'FoodNutrient', 'nutrient': {'id': 1246, 'number': '578', 'name': 'Vitamin B-12, added', 'rank': 7340, 'unitName': 'µg'}, 'id': 13209034, 'amount': 0}, {'type': 'FoodNutrient', 'nutrient': {'id': 1106, 'number': '320', 'name': 'Vitamin A, RAE', 'rank': 7420, 'unitName': 'µg'}, 'id': 13209013, 'amount': 0}, {'type': 'FoodNutrient', 'nutrient': {'id': 1105, 'number': '319', 'name': 'Retinol', 'rank': 7430, 'unitName': 'µg'}, 'id': 13209012, 'amount': 0}, {'type': 'FoodNutrient', 'nutrient': {'id': 1107, 'number': '321', 'name': 'Carotene, beta', 'rank': 7440, 'unitName': 'µg'}, 'id': 13209014, 'amount': 3}, {'type': 'FoodNutrient', 'nutrient': {'id': 1108, 'number': '322', 'name': 'Carotene, alpha', 'rank': 7450, 'unitName': 'µg'}, 'id': 13209015, 'amount': 0}, {'type': 'FoodNutrient', 'nutrient': {'id': 1120, 'number': '334', 'name': 'Cryptoxanthin, beta', 'rank': 7460, 'unitName': 'µg'}, 'id': 13209018, 'amount': 2}, {'type': 'FoodNutrient', 'nutrient': {'id': 1122, 'number': '337', 'name': 'Lycopene', 'rank': 7530, 'unitName': 'µg'}, 'id': 13209019, 'amount': 0}, {'type': 'FoodNutrient', 'nutrient': {'id': 1123, 'number': '338', 'name': 'Lutein + zeaxanthin', 'rank': 7560, 'unitName': 'µg'}, 'id': 13209020, 'amount': 18}, {'type': 'FoodNutrient', 'nutrient': {'id': 1109, 'number': '323', 'name': 'Vitamin E (alpha-tocopherol)', 'rank': 7905, 'unitName': 'mg'}, 'id': 13209016, 'amount': 0.18}, {'type': 'FoodNutrient', 'nutrient': {'id': 1242, 'number': '573', 'name': 'Vitamin E, added', 'rank': 7920, 'unitName': 'mg'}, 'id': 13209033, 'amount': 0}, {'type': 'FoodNutrient', 'nutrient': {'id': 1114, 'number': '328', 'name': 'Vitamin D (D2 + D3)', 'rank': 8700, 'unitName': 'µg'}, 'id': 13209017, 'amount': 0}, {'type': 'FoodNutrient', 'nutrient': {'id': 1185, 'number': '430', 'name': 'Vitamin K (phylloquinone)', 'rank': 8800, 'unitName': 'µg'}, 'id': 13209029, 'amount': 0.6}, {'nutrient': {'id': 2044, 'number': '950', 'name': 'Lipids', 'rank': 9600, 'unitName': 'g'}, 'type': 'FoodNutrient'}, {'type': 'FoodNutrient', 'nutrient': {'id': 1258, 'number': '606', 'name': 'Fatty acids, total saturated', 'rank': 9700, 'unitName': 'g'}, 'id': 13209036, 'amount': 0.03}, {'type': 'FoodNutrient', 'nutrient': {'id': 1259, 'number': '607', 'name': 'SFA 4:0', 'rank': 9800, 'unitName': 'g'}, 'id': 13209037, 'amount': 0}, {'type': 'FoodNutrient', 'nutrient': {'id': 1260, 'number': '608', 'name': 'SFA 6:0', 'rank': 9900, 'unitName': 'g'}, 'id': 13209038, 'amount': 0}, {'type': 'FoodNutrient', 'nutrient': {'id': 1261, 'number': '609', 'name': 'SFA 8:0', 'rank': 10000, 'unitName': 'g'}, 'id': 13209039, 'amount': 0}, {'type': 'FoodNutrient', 'nutrient': {'id': 1262, 'number': '610', 'name': 'SFA 10:0', 'rank': 10100, 'unitName': 'g'}, 'id': 13209040, 'amount': 0}, {'type': 'FoodNutrient', 'nutrient': {'id': 1263, 'number': '611', 'name': 'SFA 12:0', 'rank': 10300, 'unitName': 'g'}, 'id': 13209041, 'amount': 0.001}, {'type': 'FoodNutrient', 'nutrient': {'id': 1264, 'number': '612', 'name': 'SFA 14:0', 'rank': 10500, 'unitName': 'g'}, 'id': 13209042, 'amount': 0.001}, {'type': 'FoodNutrient', 'nutrient': {'id': 1265, 'number': '613', 'name': 'SFA 16:0', 'rank': 10700, 'unitName': 'g'}, 'id': 13209043, 'amount': 0.025}, {'type': 'FoodNutrient', 'nutrient': {'id': 1266, 'number': '614', 'name': 'SFA 18:0', 'rank': 10900, 'unitName': 'g'}, 'id': 13209044, 'amount': 0.004}, {'type': 'FoodNutrient', 'nutrient': {'id': 1292, 'number': '645', 'name': 'Fatty acids, total monounsaturated', 'rank': 11400, 'unitName': 'g'}, 'id': 13209056, 'amount': 0.007}, {'type': 'FoodNutrient', 'nutrient': {'id': 1275, 'number': '626', 'name': 'MUFA 16:1', 'rank': 11700, 'unitName': 'g'}, 'id': 13209050, 'amount': 0}, {'type': 'FoodNutrient', 'nutrient': {'id': 1268, 'number': '617', 'name': 'MUFA 18:1', 'rank': 12100, 'unitName': 'g'}, 'id': 13209045, 'amount': 0.007}, {'type': 'FoodNutrient', 'nutrient': {'id': 1277, 'number': '628', 'name': 'MUFA 20:1', 'rank': 12400, 'unitName': 'g'}, 'id': 13209052, 'amount': 0}, {'type': 'FoodNutrient', 'nutrient': {'id': 1279, 'number': '630', 'name': 'MUFA 22:1', 'rank': 12500, 'unitName': 'g'}, 'id': 13209054, 'amount': 0}, {'type': 'FoodNutrient', 'nutrient': {'id': 1293, 'number': '646', 'name': 'Fatty acids, total polyunsaturated', 'rank': 12900, 'unitName': 'g'}, 'id': 13209057, 'amount': 0.054}, {'type': 'FoodNutrient', 'nutrient': {'id': 1269, 'number': '618', 'name': 'PUFA 18:2', 'rank': 13100, 'unitName': 'g'}, 'id': 13209046, 'amount': 0.045}, {'type': 'FoodNutrient', 'nutrient': {'id': 1270, 'number': '619', 'name': 'PUFA 18:3', 'rank': 13900, 'unitName': 'g'}, 'id': 13209047, 'amount': 0.009}, {'type': 'FoodNutrient', 'nutrient': {'id': 1276, 'number': '627', 'name': 'PUFA 18:4', 'rank': 14250, 'unitName': 'g'}, 'id': 13209051, 'amount': 0}, {'type': 'FoodNutrient', 'nutrient': {'id': 1271, 'number': '620', 'name': 'PUFA 20:4', 'rank': 14700, 'unitName': 'g'}, 'id': 13209048, 'amount': 0}, {'type': 'FoodNutrient', 'nutrient': {'id': 1278, 'number': '629', 'name': 'PUFA 20:5 n-3 (EPA)', 'rank': 15000, 'unitName': 'g'}, 'id': 13209053, 'amount': 0}, {'type': 'FoodNutrient', 'nutrient': {'id': 1280, 'number': '631', 'name': 'PUFA 22:5 n-3 (DPA)', 'rank': 15200, 'unitName': 'g'}, 'id': 13209055, 'amount': 0}, {'type': 'FoodNutrient', 'nutrient': {'id': 1272, 'number': '621', 'name': 'PUFA 22:6 n-3 (DHA)', 'rank': 15300, 'unitName': 'g'}, 'id': 13209049, 'amount': 0}, {'type': 'FoodNutrient', 'nutrient': {'id': 1253, 'number': '601', 'name': 'Cholesterol', 'rank': 15700, 'unitName': 'mg'}, 'id': 13209035, 'amount': 0}, {'nutrient': {'id': 2042, 'number': '500', 'name': 'Amino acids', 'rank': 16250, 'unitName': 'g'}, 'type': 'FoodNutrient'}, {'type': 'FoodNutrient', 'nutrient': {'id': 1018, 'number': '221', 'name': 'Alcohol, ethyl', 'rank': 18200, 'unitName': 'g'}, 'id': 13208997, 'amount': 0}, {'type': 'FoodNutrient', 'nutrient': {'id': 1057, 'number': '262', 'name': 'Caffeine', 'rank': 18300, 'unitName': 'mg'}, 'id': 13208999, 'amount': 0}, {'type': 'FoodNutrient', 'nutrient': {'id': 1058, 'number': '263', 'name': 'Theobromine', 'rank': 18400, 'unitName': 'mg'}, 'id': 13209000, 'amount': 0}], 'foodPortions': [{'id': 239216, 'portionDescription': '1 cup', 'gramWeight': 250.0, 'sequenceNumber': 3, 'modifier': '10205', 'measureUnit': {'id': 9999, 'name': 'undetermined', 'abbreviation': 'undetermined'}}, {'id': 251190, 'portionDescription': '1 pouch', 'gramWeight': 90.0, 'sequenceNumber': 2, 'modifier': '61781', 'measureUnit': {'id': 9999, 'name': 'undetermined', 'abbreviation': 'undetermined'}}, {'id': 255582, 'portionDescription': '1 snack size container', 'gramWeight': 113.0, 'sequenceNumber': 1, 'modifier': '64133', 'measureUnit': {'id': 9999, 'name': 'undetermined', 'abbreviation': 'undetermined'}}, {'id': 265405, 'portionDescription': 'Quantity not specified', 'gramWeight': 125.0, 'sequenceNumber': 4, 'modifier': '90000', 'measureUnit': {'id': 9999, 'name': 'undetermined', 'abbreviation': 'undetermined'}}], 'dataType': 'Survey (FNDDS)'}

In [13]:
import requests
from google_trans_new import google_translator

# -----------------------------
# 1️⃣ 設定 API 與中文對照表
# -----------------------------
API_KEY = "YOUR_API_KEY"  # 替換成你的 USDA API key
BASE_URL = "https://api.nal.usda.gov/fdc/v1/"
translator = google_translator()

# 中文對照表（可擴充）
cname_map = {
    "Apple, raw": "蘋果",
    "Banana, raw": "香蕉",
    "Apricot, raw": "杏子",
    "Cherries, raw": "櫻桃",
    "Grapes, raw": "葡萄",
    "Spinach, raw": "菠菜",
    "Chicken, roasted": "烤雞"
}

# 需要完整的營養素
required_nutrients = [
    "Energy",
    "Protein",
    "Total lipid (fat)",
    "Carbohydrate, by difference",
    "Sugars, total including NLEA",
    "Sodium, Na",
    "Water"
]

# -----------------------------
# 2️⃣ 取得食物資料
# -----------------------------
def get_food_details(fdc_id):
    url = f"{BASE_URL}food/{fdc_id}?api_key={API_KEY}"
    response = requests.get(url)
    return response.json()

# -----------------------------
# 3️⃣ 檢查營養素完整
# -----------------------------
def has_all_nutrients(food_json):
    names = [n.get('nutrient', {}).get('name') or n.get('nutrientName') for n in food_json.get('foodNutrients', [])]
    return all(r in names for r in required_nutrients)

# -----------------------------
# 4️⃣ 解析營養素
# -----------------------------
def parse_nutrients(food_json):
    nutrient_map = {
        "Energy": "energy_kcal",
        "Protein": "protein",
        "Total lipid (fat)": "fat",
        "Carbohydrate, by difference": "carbs",
        "Sugars, total including NLEA": "sugar",
        "Sodium, Na": "sodium",
        "Water": "water"
    }

    data = {}
    for nutrient in food_json.get('foodNutrients', []):
        if 'nutrient' in nutrient:
            name = nutrient['nutrient'].get('name')
            value = nutrient.get('amount')
        else:
            name = nutrient.get('nutrientName')
            value = nutrient.get('value')

        if name in nutrient_map:
            # 將鈉從 mg 轉為 g
            if name == "Sodium, Na" and value is not None:
                value = value / 1000
            data[nutrient_map[name]] = value
    return data

# -----------------------------
# 5️⃣ 生成 MySQL INSERT 語句
# -----------------------------
def generate_insert_sql(food_json, nutrients):
    ingredient_name = food_json['description'].replace("'", "''")
    # 中文名稱優先對照表，若沒有就自動翻譯
    c_name = cname_map.get(food_json['description'])
    if not c_name:
        c_name = translator.translate(food_json['description'], lang_tgt='zh-tw')
    c_name = c_name.replace("'", "''")

    insert_sql = f"""
INSERT INTO ingredient (ingredient_name, c_name, energy_kcal, protein, fat, carbs, sugar, sodium, water)
VALUES ('{ingredient_name}', '{c_name}',
        {nutrients.get('energy_kcal') if nutrients.get('energy_kcal') is not None else 'NULL'},
        {nutrients.get('protein') if nutrients.get('protein') is not None else 'NULL'},
        {nutrients.get('fat') if nutrients.get('fat') is not None else 'NULL'},
        {nutrients.get('carbs') if nutrients.get('carbs') is not None else 'NULL'},
        {nutrients.get('sugar') if nutrients.get('sugar') is not None else 'NULL'},
        {nutrients.get('sodium') if nutrients.get('sodium') is not None else 'NULL'},
        {nutrients.get('water') if nutrients.get('water') is not None else 'NULL'});
"""
    return insert_sql

# -----------------------------
# 6️⃣ 批量抓取多筆食物
# -----------------------------
fdc_ids = [
    1102645,  # Apple, raw
    1102650,  # Banana, raw
    1102660,  # Chicken, roasted
    1102665   # Spinach, raw
]

for fdc_id in fdc_ids:
    food_json = get_food_details(fdc_id)

    # 過濾非 100g 基準的食材
    if food_json.get('dataType') in ['Branded']:
        print(f"跳過 {food_json.get('description', 'Unknown Food')}，非 100g 基準")
        continue

    # 過濾營養素不完整的食材
    if not has_all_nutrients(food_json):
        print(f"跳過 {food_json.get('description', 'Unknown Food')}，營養素不完整")
        continue

    nutrients = parse_nutrients(food_json)
    sql = generate_insert_sql(food_json, nutrients)
    print(sql)



跳過 Unknown Food，營養素不完整
跳過 Unknown Food，營養素不完整
跳過 Unknown Food，營養素不完整
跳過 Unknown Food，營養素不完整
